In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from scipy.stats import percentileofscore
import numpy as np

ModuleNotFoundError: No module named 'bs4'

In [ ]:
# Don't need to f-string loop this, all column headers will remain the same
source = requests.get('https://www.fangraphs.com/leaders.aspx?pos=all&stats=sta&lg=all&qual=y&type=c,4,5,11,7,8,13,-1,36,37,40,43,44,48,51,-1,6,45,62,-1,59,3,21,47,49,60,75,77,79,81,83,85,87,76,41,42&season=2018&month=0&season1=2000&ind=1&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=').text
soup = BeautifulSoup(source, 'lxml')
cols = []
matches = soup.find_all('th', scope = 'col')
for match in matches:
    match = str(match)
    match = match[match.find(',d">')+len(',d">'):match.find('</a>')]
    cols.append(match)

cols[0] = cols[0][cols[0].rfind('>')+1:cols[0].find('<')]
cols[20] = cols[20][cols[20].rfind('>')+1:]
cols[24] = 'FlyBall%'
cols

In [ ]:
dfs = []
for n in range(1,123):
    page = requests.get(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=sta&lg=all&qual=0&type=c,4,5,11,7,8,13,-1,36,37,40,43,44,48,51,-1,6,45,62,-1,59,3,21,47,49,60,75,77,79,81,83,85,87,76,41,42&season=2019&month=0&season1=2000&ind=1&team=0&rost=0&age=0&filter=&players=0&startdate=2000-01-01&enddate=2019-12-31&page={n}_50').content
    df = pd.read_html(page)
    # Remove the last "next page" row from the dataframe
    players = df[16].iloc[:-1, :]
    players.columns = cols
    dfs.append(players)
    time.sleep(3)

In [206]:
combined = pd.DataFrame()
for data in dfs:
    combined = pd.concat([combined, data], sort = False)
    

In [207]:
# combined.to_csv('Combined_uncleaned.csv')

In [208]:
combined.drop(['#', 'Dollars'], axis = 1, inplace = True)

In [209]:
combined.sort_values(by=['Name', 'Season'], inplace = True)

In [210]:
combined.reset_index(drop = True, inplace = True)

In [211]:
combined['Name'].value_counts() == 1

CC Sabathia      False
Bartolo Colon    False
Kyle Lohse       False
Mark Buehrle     False
Tim Hudson       False
                 ...  
Neftali Feliz     True
Tony Pena         True
Brett Kennedy     True
Andrew Werner     True
Peter Lambert     True
Name: Name, Length: 1668, dtype: bool

In [212]:
combined['Name'].value_counts()

CC Sabathia      19
Bartolo Colon    18
Kyle Lohse       16
Mark Buehrle     16
Tim Hudson       16
                 ..
Neftali Feliz     1
Tony Pena         1
Brett Kennedy     1
Andrew Werner     1
Peter Lambert     1
Name: Name, Length: 1668, dtype: int64

In [213]:
combined = combined.groupby('Name').filter(lambda x : len(x)>3)

In [274]:
combined['Name'].value_counts()

CC Sabathia         18
Edwin Jackson       16
Bartolo Colon       16
Justin Verlander    15
Zack Greinke        15
                    ..
Ryan Glynn           2
James Baldwin        2
Denny Neagle         2
Ryan Rupe            2
John Parrish         1
Name: Name, Length: 595, dtype: int64

In [214]:
combined['Season'] = combined['Season'].astype('int')

In [216]:
combined.drop(combined[combined['Season'] < 2002].index, inplace = True)

In [217]:
combined.drop(combined[combined['FB%'].isna()].index, inplace = True)

In [221]:
combined.drop(combined[combined['FBv'].isna()].index, inplace = True)

In [222]:
combined.isna().sum()

Season         0
Name           0
Team           0
W              0
L              0
SV             0
G              0
GS             0
IP             0
K/9            0
BB/9           0
HR/9           0
BABIP          0
LOB%           0
GB%            0
HR/FB          0
ERA            0
FIP            0
xFIP           0
WAR            0
Age            0
HBP            0
LD%            0
FlyBall%       0
FB%            0
SL%          879
CT%         2594
CB%          518
CH%          184
SF%         3408
KN%         3908
FBv            0
AVG            0
WHIP           0
dtype: int64

In [223]:
combined.fillna(0, inplace = True)

In [224]:
combined.isna().sum()

Season      0
Name        0
Team        0
W           0
L           0
SV          0
G           0
GS          0
IP          0
K/9         0
BB/9        0
HR/9        0
BABIP       0
LOB%        0
GB%         0
HR/FB       0
ERA         0
FIP         0
xFIP        0
WAR         0
Age         0
HBP         0
LD%         0
FlyBall%    0
FB%         0
SL%         0
CT%         0
CB%         0
CH%         0
SF%         0
KN%         0
FBv         0
AVG         0
WHIP        0
dtype: int64

In [225]:
combined.reset_index(drop = True, inplace = True)

In [226]:
# Strip % and $ signs from all columns
for column in combined.columns[1:]:
    combined[column] = combined[column].str.strip('%')
    combined[column] = combined[column].str.strip('$')

In [227]:
for colname in combined.columns[3:]:
    combined[colname] = pd.to_numeric(combined[colname], errors = 'coerce')

In [228]:
combined['IP/G']  = round(combined['IP']//combined['GS'] + (combined['IP']/combined['GS'] - combined['IP']//combined['GS']), 2)
combined['IP/G']

0       6.94
1       5.75
2       6.21
3       6.53
4       6.44
        ... 
3949    5.88
3950    5.78
3951    5.06
3952    6.28
3953    6.29
Name: IP/G, Length: 3954, dtype: float64

In [80]:
# K_ptile = []
# BB_ptile = []
# IPG_ptile = []
# BABIP_ptile = []
# FBv_ptile = []
# GB_ptile = []
# for i in range(len(combined)):
#     K_ptile.append(percentileofscore(combined['K/9'], combined.loc[i,'K/9']))
#     BB_ptile.append(100 - (percentileofscore(combined['BB/9'], combined.loc[i,'BB/9'])))
#     IPG_ptile.append(percentileofscore(combined['IP/G'], combined.loc[i,'IP/G']))
#     BABIP_ptile.append(100-(percentileofscore(combined['BABIP'], combined.loc[i,'BABIP'])))
#     FBv_ptile.append(percentileofscore(combined['FBv'], combined.loc[i,'FBv']))
#     GB_ptile.append(percentileofscore(combined['GB%'], combined.loc[i,'GB%']))
# combined['Strikeouts'] = K_ptile
# combined['Control'] = BB_ptile
# combined['Stamina/Consistency']= IPG_ptile
# combined['BA Against'] = BABIP_ptile
# combined['Velocity'] = FBv_ptile
# combined['Ground Balls'] = GB_ptile

C:\ProgramData\Anaconda\lib\site-packages\scipy\stats\stats.py:1857: RuntimeWarning: invalid value encountered in less
  left = np.count_nonzero(a < score)
C:\ProgramData\Anaconda\lib\site-packages\scipy\stats\stats.py:1858: RuntimeWarning: invalid value encountered in less_equal
  right = np.count_nonzero(a <= score)
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [229]:
# Diversity is based off of Shannon's Diversity Index D = sum(pi * ln(pi))
pitchdf = pd.DataFrame(
    ((combined.loc[:,'FB%':'KN%']/100).loc[:,:]*np.log((combined.loc[:,'FB%':'KN%']/100).loc[:,:])).fillna(0))
pitchdf['diversity'] = pitchdf.sum(axis=1)*(-1)
pitchdf

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


,FB%,SL%,CT%,CB%,CH%,SF%,KN%,diversity
0,-0.252234,0.000000,0.0,-0.347338,-0.153739,0.000000,0.0,0.753311
1,-0.223516,-0.046052,0.0,-0.304274,-0.198982,0.000000,0.0,0.772823
2,-0.235851,-0.117334,0.0,-0.321888,-0.137437,0.000000,0.0,0.812509
3,-0.287826,-0.017427,0.0,-0.350926,-0.228951,0.000000,0.0,0.885130
4,-0.268320,0.000000,0.0,-0.358501,-0.133144,0.000000,0.0,0.759964
...,...,...,...,...,...,...,...,...
3949,-0.242510,-0.290687,0.0,-0.230259,-0.112571,0.000000,0.0,0.876027
3950,-0.295859,-0.284568,0.0,-0.291535,-0.189438,0.000000,0.0,1.061401
3951,-0.297423,-0.323096,0.0,-0.270335,-0.141639,0.000000,0.0,1.032493
3952,-0.315028,-0.316850,0.0,-0.236648,-0.133144,-0.198982,0.0,1.200651


In [236]:
combined['Shannon Diversity'] = pitchdf['diversity']

In [231]:
combined

,Season,Name,Team,W,L,SV,G,GS,IP,K/9,...,CT%,CB%,CH%,SF%,KN%,FBv,AVG,WHIP,IP/G,Pitch Diversity
0,2002,A.J. Burnett,Marlins,12,9,0,29,29,201.2,8.70,...,NaN,25.2,5.2,NaN,NaN,94.4,0.202,1.17,6.94,0.753311
1,2003,A.J. Burnett,Marlins,0,2,0,4,4,23.0,8.22,...,NaN,17.4,7.8,NaN,NaN,92.8,0.209,1.57,5.75,0.772823
2,2004,A.J. Burnett,Marlins,7,6,0,19,19,118.0,8.31,...,NaN,20.0,4.4,NaN,NaN,95.4,0.229,1.18,6.21,0.812509
3,2005,A.J. Burnett,Marlins,12,12,0,32,32,209.0,8.53,...,NaN,26.2,9.9,NaN,NaN,95.6,0.234,1.26,6.53,0.885130
4,2006,A.J. Burnett,Blue Jays,10,8,0,21,21,135.2,7.83,...,NaN,28.8,4.2,NaN,NaN,94.9,0.260,1.30,6.44,0.759964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3949,2013,Zack Wheeler,Mets,7,5,0,17,17,100.0,7.56,...,NaN,10.0,3.3,NaN,NaN,94.4,0.236,1.36,5.88,0.876027
3950,2014,Zack Wheeler,Mets,11,11,0,32,32,185.1,9.08,...,NaN,15.8,7.2,NaN,NaN,95.0,0.237,1.33,5.78,1.061401
3951,2017,Zack Wheeler,Mets,3,7,0,17,17,86.1,8.44,...,NaN,13.5,4.6,NaN,NaN,94.6,0.283,1.59,5.06,1.032493
3952,2018,Zack Wheeler,Mets,12,7,0,29,29,182.1,8.84,...,NaN,10.5,4.2,7.8,NaN,95.9,0.221,1.12,6.28,1.200651


In [237]:
combined.to_csv('combined_cleaned.csv')

In [238]:
year_dfs = []
for year in pd.unique(combined['Season']):
    year_dfs.append(combined[combined['Season'] == year])

In [239]:
year_dfs[0]

,Season,Name,Team,W,L,SV,G,GS,IP,K/9,...,CB%,CH%,SF%,KN%,FBv,AVG,WHIP,IP/G,Pitch Diversity,Shannon Diversity
0,2002,A.J. Burnett,Marlins,12,9,0,29,29,201.2,8.70,...,25.2,5.2,NaN,NaN,94.4,0.202,1.17,6.94,0.753311,0.753311
22,2002,Aaron Cook,Rockies,2,1,0,5,5,30.0,3.60,...,NaN,6.1,NaN,NaN,92.3,0.308,1.57,6.00,0.819481,0.819481
33,2002,Aaron Harang,Athletics,5,3,0,15,15,76.1,7.43,...,NaN,9.6,0.1,NaN,84.0,0.254,1.55,5.07,0.745803,0.745803
63,2002,Aaron Sele,Angels,8,9,0,26,26,160.0,4.61,...,31.8,8.5,NaN,NaN,85.4,0.292,1.49,6.15,1.069896,1.069896
68,2002,Adam Bernero,Tigers,2,6,0,11,11,64.1,5.88,...,NaN,21.9,8.4,NaN,90.1,0.289,1.52,5.83,1.094018,1.094018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,2002,Wade Miller,Astros,15,4,0,26,26,164.2,7.87,...,18.1,3.7,NaN,NaN,92.5,0.244,1.29,6.32,0.898742,0.898742
3836,2002,Wes Obermueller,Royals,0,2,0,2,2,7.2,5.87,...,NaN,14.5,NaN,NaN,88.3,0.378,2.09,3.60,0.837589,0.837589
3847,2002,Wilson Alvarez,Devil Rays,2,2,0,10,10,47.0,5.94,...,16.3,10.2,NaN,NaN,86.3,0.289,1.68,4.70,0.754822,0.754822
3857,2002,Woody Williams,Cardinals,9,4,0,17,17,103.1,6.62,...,17.1,8.3,NaN,NaN,88.0,0.219,1.05,6.06,0.845891,0.845891


In [253]:
for year_df in year_dfs:
    year_df.reset_index(drop = True, inplace = True)
    K_ptile = []
    BB_ptile = []
    IPG_ptile = []
    BABIP_ptile = []
    FBv_ptile = []
    GB_ptile = []
    div_ptile = []
    for i in range(len(year_df)):     
        K_ptile.append(percentileofscore(year_df['K/9'], year_df.loc[i,'K/9']))
        BB_ptile.append(100 - (percentileofscore(year_df['BB/9'], year_df.loc[i,'BB/9'])))
        IPG_ptile.append(percentileofscore(year_df['IP/G'], year_df.loc[i,'IP/G']))
        BABIP_ptile.append(100-(percentileofscore(year_df['BABIP'], year_df.loc[i,'BABIP'])))
        FBv_ptile.append(percentileofscore(year_df['FBv'], year_df.loc[i,'FBv']))
        GB_ptile.append(percentileofscore(year_df['GB%'], year_df.loc[i,'GB%']))
        div_ptile.append(percentileofscore(year_df['Shannon Diversity'], year_df.loc[i,'Shannon Diversity']))
    year_df['Strikeouts'] = K_ptile
    year_df['Control'] = BB_ptile
    year_df['Stamina/Consistency']= IPG_ptile
    year_df['BA Against'] = BABIP_ptile
    year_df['Velocity'] = FBv_ptile
    year_df['Ground Balls'] = GB_ptile
    year_df['Pitch Diversity'] = div_ptile

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [254]:
final = pd.DataFrame()
for year_df in year_dfs:
    final = pd.concat([final, year_df], sort = False)

In [255]:
final.sort_values(by=['Name','Season'], inplace = True)

In [256]:
final.reset_index(drop = True, inplace = True)

In [259]:
final.drop(['index'], axis=1, inplace = True)

In [260]:
final

,Season,Name,Team,W,L,SV,G,GS,IP,K/9,...,WHIP,IP/G,Pitch Diversity,Shannon Diversity,Strikeouts,Control,Stamina/Consistency,BA Against,Velocity,Ground Balls
0,2002,A.J. Burnett,Marlins,12,9,0,29,29,201.2,8.70,...,1.17,6.94,14.500000,0.753311,91.000000,27.750000,96.250000,88.500000,100.000000,57.500000
1,2003,A.J. Burnett,Marlins,0,2,0,4,4,23.0,8.22,...,1.57,5.75,8.755760,0.772823,90.783410,3.686636,51.843318,86.635945,97.004608,88.940092
2,2004,A.J. Burnett,Marlins,7,6,0,19,19,118.0,8.31,...,1.18,6.21,9.734513,0.812509,89.823009,63.716814,78.982301,61.725664,100.000000,80.309735
3,2005,A.J. Burnett,Marlins,12,12,0,32,32,209.0,8.53,...,1.26,6.53,20.614035,0.885130,92.543860,37.280702,86.403509,40.789474,99.122807,94.736842
4,2006,A.J. Burnett,Blue Jays,10,8,0,21,21,135.2,7.83,...,1.30,6.44,7.203390,0.759964,82.415254,73.305085,87.288136,33.686441,98.728814,85.381356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3949,2013,Zack Wheeler,Mets,7,5,0,17,17,100.0,7.56,...,1.36,5.88,15.966387,0.876027,64.285714,14.705882,56.722689,71.218487,92.226891,46.428571
3950,2014,Zack Wheeler,Mets,11,11,0,32,32,185.1,9.08,...,1.33,5.78,50.643777,1.061401,86.695279,18.884120,47.854077,38.626609,96.566524,91.845494
3951,2017,Zack Wheeler,Mets,3,7,0,17,17,86.1,8.44,...,1.59,5.06,37.313433,1.032493,67.164179,14.925373,27.114428,16.666667,87.810945,71.890547
3952,2018,Zack Wheeler,Mets,12,7,0,29,29,182.1,8.84,...,1.12,6.28,68.208092,1.200651,67.052023,59.537572,96.242775,69.942197,95.086705,55.491329


In [261]:
final.to_csv('Master_df.csv')

In [273]:
final[final['Name']=='Felix Hernandez']

,Season,Name,Team,W,L,SV,G,GS,IP,K/9,...,WHIP,IP/G,Pitch Diversity,Shannon Diversity,Strikeouts,Control,Stamina/Consistency,BA Against,Velocity,Ground Balls
1272,2005,Felix Hernandez,Mariners,4,4,0,12,12,84.1,8.22,...,1.00,7.01,54.824561,1.033015,88.157895,72.368421,97.587719,89.692982,99.561404,99.561404
1273,2006,Felix Hernandez,Mariners,12,14,0,31,31,191.0,8.29,...,1.34,6.16,71.186441,1.118802,90.042373,65.254237,74.788136,33.686441,99.576271,94.067797
1274,2007,Felix Hernandez,Mariners,14,7,0,30,30,190.1,7.80,...,1.38,6.34,71.487603,1.134238,84.504132,75.826446,87.603306,20.041322,99.586777,96.694215
1275,2008,Felix Hernandez,Mariners,9,11,0,31,31,200.2,7.85,...,1.39,6.46,55.701754,1.010428,81.578947,35.526316,92.105263,28.508772,98.684211,89.473684
1276,2009,Felix Hernandez,Mariners,19,5,0,34,34,238.2,8.18,...,1.14,7.01,56.540084,1.060986,82.067511,75.316456,99.156118,71.940928,93.881857,91.561181
1277,2010,Felix Hernandez,Mariners,13,12,0,34,34,249.2,8.36,...,1.06,7.33,55.813953,1.097712,83.488372,72.325581,99.069767,84.883721,96.279070,92.325581
1278,2011,Felix Hernandez,Mariners,14,14,0,33,33,233.2,8.55,...,1.22,7.07,72.072072,1.176938,87.612613,61.936937,96.846847,32.207207,88.288288,80.405405
1279,2012,Felix Hernandez,Mariners,13,9,0,33,33,232.0,8.65,...,1.14,7.03,67.234043,1.165229,82.978723,77.659574,98.510638,34.042553,70.851064,72.340426
1280,2013,Felix Hernandez,Mariners,12,10,0,31,31,204.1,9.51,...,1.13,6.58,73.109244,1.175099,95.378151,84.453782,93.697479,29.411765,63.655462,84.033613
1281,2014,Felix Hernandez,Mariners,15,6,0,34,34,236.0,9.46,...,0.92,6.94,85.836910,1.228602,91.201717,86.051502,98.283262,91.416309,73.605150,96.781116
